In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook
from webdriver_manager.chrome import ChromeDriverManager
from riotwatcher import LolWatcher, ApiError

import selenium.webdriver.support.ui as ui
import time
import pandas as pd
import numpy as np
import os
import re
import threading

In [ ]:
# selenium driver
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
    {
        'notifications':2
    }
}
options.add_experimental_option('prefs', prefs)
options.add_argument("disable-infobars")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
wait = ui.WebDriverWait(driver,5)

In [ ]:
# team
divisions = ['LCS', 'LEC', 'LCK', 'LPL', 'LMS', 'CBLOL', 'LCL', 'LJL', 'LLN', 'CLS', 'OPL', 'GPL', 'TCL', 'VCS']
seasons = ['2018_Season', '2019_Season', '2020_Season']
team_names = []
team_wins = []
team_loses = []
team_winrates = []
team_streaks = []
team_divisions = []
team_seasons = []

for d in divisions:
    for s in seasons:
        website = "https://lol.gamepedia.com/{}/{}".format(d, s)
        driver.get(website)
        teams_info = driver.find_elements(By.XPATH, "//div[@class='inline-content'][1]//table[@class='wikitable2 standings']//tr")
        
        for t_f in teams_info:
            team_name = t_f.get_attribute('data-teamhighlight')
            if team_name != None:
                team_names.append(team_name)
                team_divisions.append(d)
                team_seasons.append(s.split('_')[0])

team_df = pd.DataFrame(zip(team_divisions, team_seasons, team_names), columns=['Division', 'Season', 'Team'])
team_df.drop_duplicates('Team', inplace = True)
team_df.to_excel("teams.xlsx")

In [ ]:
# player
unique_team_names = set(team_names)
player_names = []
u_team_names = []

for t_n in unique_team_names:
    website = "https://lol.gamepedia.com/{}".format(t_n.replace(' ','_'))
    driver.get(website)
    time.sleep(1)
    players_info = driver.find_elements(By.XPATH, "//table[@class='wikitable sortable team-members hoverable-rows team-members-current jquery-tablesorter']//tr")

    for p_i in players_info:
        try:
            print(p_i.text.split('\n')[1].split(' ')[0])
            player_name = p_i.text.split('\n')[1].split(' ')[0]
            player_names.append(player_name)
            u_team_names.append(t_n)
            
        except:
            pass

player_df = pd.DataFrame(zip(u_team_names, player_names), columns=['Team', 'Player'])
player_df.to_excel('player.xlsx')

In [ ]:
player_team_df = pd.merge(team_df, player_df, how='right', on='Team')
player_team_df.to_excel("player_team.xlsx")

In [ ]:
# read player_team table
player_team = pd.read_excel('player_team.xlsx')
player_team = player_team.drop('Unnamed: 0', axis=1)
player_team

In [ ]:
server_division_dict = {"LCS":"na1", "LEC":"eun1", "LEC":"euw1", "LCK":"kr", "LPL":"jp1", "LMS":"jp1", "CBLOL":"br1", "LCL":"ru", "LJL":"jp1",
                        "LLA":"la1", "LLA":"la2", "OPL":"oc1", "TCL":"tr1"}

In [ ]:
# get player matches info
api_key = 'RGAPI-0be7ad8b-756d-4b7a-978b-9aa75f81b2b3'
watcher = LolWatcher(api_key)
part_data= round(len(player_team)/5)

matches_info = []
for i in range(part_data, part_data*2):

    region = server_division_dict[player_team['Division'][i]]
    player_name = player_team['Player'][i]
    print(region)
    print(player_name)
    print(i)
    
    try:
        player = watcher.summoner.by_name(region, player_name)
        print(player)
        player_matches = watcher.match.matchlist_by_account(region, player['accountId'])['matches']

        j=0
        for player_match in player_matches:
            print('match----',j)
            j = j+1
            info_row = {}
            try:
                m = watcher.match.by_id(region, player_match['gameId'])
                info_row['Player_name'] = player_name
                info_row['gameId'] = m['gameId']
                info_row['platformId'] = m['platformId']
                info_row['gameDuration'] = m['gameDuration']
                info_row['gameVersion'] = m['gameVersion']

                blue_team = m['teams'][0]
                purple_team = m['teams'][1]
                info_row['blue_win'] = blue_team['win']
                info_row['blue_firstBlood'] = blue_team['firstBlood']
                info_row['blue_firstTower'] = blue_team['firstTower']
                info_row['blue_firstInhibitor'] = blue_team['firstInhibitor']
                info_row['blue_firstBaron'] = blue_team['firstBaron']
                info_row['blue_firstDragon'] = blue_team['firstDragon']
                info_row['blue_firstRiftHerald'] = blue_team['firstRiftHerald']
                info_row['blue_towerKills'] = blue_team['towerKills']
                info_row['blue_inhibitorKills'] = blue_team['inhibitorKills']
                info_row['blue_baronKills'] = blue_team['baronKills']
                info_row['blue_dragonKills'] = blue_team['dragonKills']
                info_row['blue_riftHeraldKills'] = blue_team['riftHeraldKills']

                info_row['purple_firstBlood'] = purple_team['firstBlood']
                info_row['purple_firstTower'] = purple_team['firstTower']
                info_row['purple_firstInhibitor'] = purple_team['firstInhibitor']
                info_row['purple_firstBaron'] = purple_team['firstBaron']
                info_row['purple_firstDragon'] = purple_team['firstDragon']
                info_row['purple_firstRiftHerald'] = purple_team['firstRiftHerald']
                info_row['purple_towerKills'] = purple_team['towerKills']
                info_row['purple_inhibitorKills'] = purple_team['inhibitorKills']
                info_row['purple_baronKills'] = purple_team['baronKills']
                info_row['purple_dragonKills'] = purple_team['dragonKills']
                info_row['purple_riftHeraldKills'] = purple_team['riftHeraldKills']

                ps = 0
                participants = m['participants']
                for p in participants:
                    print(ps)
                    ps = ps+1
                    if p['teamId'] == 100:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['blue_top_champion'] = p['championId']
                            info_row['blue_top_item0'] = p['stats']['item0']
                            info_row['blue_top_item1'] = p['stats']['item1']
                            info_row['blue_top_item2'] = p['stats']['item2']
                            info_row['blue_top_item3'] = p['stats']['item3']
                            info_row['blue_top_item4'] = p['stats']['item4']
                            info_row['blue_top_item5'] = p['stats']['item5']
                            info_row['blue_top_item6'] = p['stats']['item6']
                            info_row['blue_top_kills'] = p['stats']['kills']
                            info_row['blue_top_deaths'] = p['stats']['deaths']
                            info_row['blue_top_assists'] = p['stats']['assists']
        #                     info_row['blue_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_top_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_top_perk0'] = p['stats']['perk0']
                            info_row['blue_top_perk1'] = p['stats']['perk1']
                            info_row['blue_top_perk2'] = p['stats']['perk2']
                            info_row['blue_top_perk3'] = p['stats']['perk3']
                            info_row['blue_top_perk4'] = p['stats']['perk4']
                            info_row['blue_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['blue_jungle_champion'] = p['championId']
                            info_row['blue_jungle_item0'] = p['stats']['item0']
                            info_row['blue_jungle_item1'] = p['stats']['item1']
                            info_row['blue_jungle_item2'] = p['stats']['item2']
                            info_row['blue_jungle_item3'] = p['stats']['item3']
                            info_row['blue_jungle_item4'] = p['stats']['item4']
                            info_row['blue_jungle_item5'] = p['stats']['item5']
                            info_row['blue_jungle_item6'] = p['stats']['item6']
                            info_row['blue_jungle_kills'] = p['stats']['kills']
                            info_row['blue_jungle_deaths'] = p['stats']['deaths']
                            info_row['blue_jungle_assists'] = p['stats']['assists']
        #                     info_row['blue_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
    #                         info_row['blue_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
    #                         info_row['blue_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
    #                         info_row['blue_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_jungle_perk0'] = p['stats']['perk0']
                            info_row['blue_jungle_perk1'] = p['stats']['perk1']
                            info_row['blue_jungle_perk2'] = p['stats']['perk2']
                            info_row['blue_jungle_perk3'] = p['stats']['perk3']
                            info_row['blue_jungle_perk4'] = p['stats']['perk4']
                            info_row['blue_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['blue_mid_champion'] = p['championId']
                            info_row['blue_mid_item0'] = p['stats']['item0']
                            info_row['blue_mid_item1'] = p['stats']['item1']
                            info_row['blue_mid_item2'] = p['stats']['item2']
                            info_row['blue_mid_item3'] = p['stats']['item3']
                            info_row['blue_mid_item4'] = p['stats']['item4']
                            info_row['blue_mid_item5'] = p['stats']['item5']
                            info_row['blue_mid_item6'] = p['stats']['item6']
                            info_row['blue_mid_kills'] = p['stats']['kills']
                            info_row['blue_mid_deaths'] = p['stats']['deaths']
                            info_row['blue_mid_assists'] = p['stats']['assists']
        #                     info_row['blue_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_mid_perk0'] = p['stats']['perk0']
                            info_row['blue_mid_perk1'] = p['stats']['perk1']
                            info_row['blue_mid_perk2'] = p['stats']['perk2']
                            info_row['blue_mid_perk3'] = p['stats']['perk3']
                            info_row['blue_mid_perk4'] = p['stats']['perk4']
                            info_row['blue_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['blue_ad_champion'] = p['championId']
                            info_row['blue_ad_item0'] = p['stats']['item0']
                            info_row['blue_ad_item1'] = p['stats']['item1']
                            info_row['blue_ad_item2'] = p['stats']['item2']
                            info_row['blue_ad_item3'] = p['stats']['item3']
                            info_row['blue_ad_item4'] = p['stats']['item4']
                            info_row['blue_ad_item5'] = p['stats']['item5']
                            info_row['blue_ad_item6'] = p['stats']['item6']
                            info_row['blue_ad_kills'] = p['stats']['kills']
                            info_row['blue_ad_deaths'] = p['stats']['deaths']
                            info_row['blue_ad_assists'] = p['stats']['assists']
        #                     info_row['blue_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['blue_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['blue_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['blue_ad_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['blue_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_ad_perk0'] = p['stats']['perk0']
                            info_row['blue_ad_perk1'] = p['stats']['perk1']
                            info_row['blue_ad_perk2'] = p['stats']['perk2']
                            info_row['blue_ad_perk3'] = p['stats']['perk3']
                            info_row['blue_ad_perk4'] = p['stats']['perk4']
                            info_row['blue_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['blue_sup_champion'] = p['championId']
                            info_row['blue_sup_item0'] = p['stats']['item0']
                            info_row['blue_sup_item1'] = p['stats']['item1']
                            info_row['blue_sup_item2'] = p['stats']['item2']
                            info_row['blue_sup_item3'] = p['stats']['item3']
                            info_row['blue_sup_item4'] = p['stats']['item4']
                            info_row['blue_sup_item5'] = p['stats']['item5']
                            info_row['blue_sup_item6'] = p['stats']['item6']
                            info_row['blue_sup_kills'] = p['stats']['kills']
                            info_row['blue_sup_deaths'] = p['stats']['deaths']
                            info_row['blue_sup_assists'] = p['stats']['assists']
        #                     info_row['blue_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']                   
        #                     info_row['blue_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']                 
        #                     info_row['blue_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_sup_perk0'] = p['stats']['perk0']
                            info_row['blue_sup_perk1'] = p['stats']['perk1']
                            info_row['blue_sup_perk2'] = p['stats']['perk2']
                            info_row['blue_sup_perk3'] = p['stats']['perk3']
                            info_row['blue_sup_perk4'] = p['stats']['perk4']
                            info_row['blue_sup_perk5'] = p['stats']['perk5']

                    if p['teamId'] == 200:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['purple_top_champion'] = p['championId']
                            info_row['purple_top_item0'] = p['stats']['item0']
                            info_row['purple_top_item1'] = p['stats']['item1']
                            info_row['purple_top_item2'] = p['stats']['item2']
                            info_row['purple_top_item3'] = p['stats']['item3']
                            info_row['purple_top_item4'] = p['stats']['item4']
                            info_row['purple_top_item5'] = p['stats']['item5']
                            info_row['purple_top_item6'] = p['stats']['item6']
                            info_row['purple_top_kills'] = p['stats']['kills']
                            info_row['purple_top_deaths'] = p['stats']['deaths']
                            info_row['purple_top_assists'] = p['stats']['assists']
        #                     info_row['purple_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['purple_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['purple_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['purple_top_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['purple_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_top_perk0'] = p['stats']['perk0']
                            info_row['purple_top_perk1'] = p['stats']['perk1']
                            info_row['purple_top_perk2'] = p['stats']['perk2']
                            info_row['purple_top_perk3'] = p['stats']['perk3']
                            info_row['purple_top_perk4'] = p['stats']['perk4']
                            info_row['purple_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['purple_jungle_champion'] = p['championId']
                            info_row['purple_jungle_item0'] = p['stats']['item0']
                            info_row['purple_jungle_item1'] = p['stats']['item1']
                            info_row['purple_jungle_item2'] = p['stats']['item2']
                            info_row['purple_jungle_item3'] = p['stats']['item3']
                            info_row['purple_jungle_item4'] = p['stats']['item4']
                            info_row['purple_jungle_item5'] = p['stats']['item5']
                            info_row['purple_jungle_item6'] = p['stats']['item6']
                            info_row['purple_jungle_kills'] = p['stats']['kills']
                            info_row['purple_jungle_deaths'] = p['stats']['deaths']
                            info_row['purple_jungle_assists'] = p['stats']['assists']
        #                     info_row['purple_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_jungle_perk0'] = p['stats']['perk0']
                            info_row['purple_jungle_perk1'] = p['stats']['perk1']
                            info_row['purple_jungle_perk2'] = p['stats']['perk2']
                            info_row['purple_jungle_perk3'] = p['stats']['perk3']
                            info_row['purple_jungle_perk4'] = p['stats']['perk4']
                            info_row['purple_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['purple_mid_champion'] = p['championId']
                            info_row['purple_mid_item0'] = p['stats']['item0']
                            info_row['purple_mid_item1'] = p['stats']['item1']
                            info_row['purple_mid_item2'] = p['stats']['item2']
                            info_row['purple_mid_item3'] = p['stats']['item3']
                            info_row['purple_mid_item4'] = p['stats']['item4']
                            info_row['purple_mid_item5'] = p['stats']['item5']
                            info_row['purple_mid_item6'] = p['stats']['item6']
                            info_row['purple_mid_kills'] = p['stats']['kills']
                            info_row['purple_mid_deaths'] = p['stats']['deaths']
                            info_row['purple_mid_assists'] = p['stats']['assists']
        #                     info_row['purple_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_mid_perk0'] = p['stats']['perk0']
                            info_row['purple_mid_perk1'] = p['stats']['perk1']
                            info_row['purple_mid_perk2'] = p['stats']['perk2']
                            info_row['purple_mid_perk3'] = p['stats']['perk3']
                            info_row['purple_mid_perk4'] = p['stats']['perk4']
                            info_row['purple_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['purple_ad_champion'] = p['championId']
                            info_row['purple_ad_item0'] = p['stats']['item0']
                            info_row['purple_ad_item1'] = p['stats']['item1']
                            info_row['purple_ad_item2'] = p['stats']['item2']
                            info_row['purple_ad_item3'] = p['stats']['item3']
                            info_row['purple_ad_item4'] = p['stats']['item4']
                            info_row['purple_ad_item5'] = p['stats']['item5']
                            info_row['purple_ad_item6'] = p['stats']['item6']
                            info_row['purple_ad_kills'] = p['stats']['kills']
                            info_row['purple_ad_deaths'] = p['stats']['deaths']
                            info_row['purple_ad_assists'] = p['stats']['assists']
        #                     info_row['purple_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_ad_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_ad_perk0'] = p['stats']['perk0']
                            info_row['purple_ad_perk1'] = p['stats']['perk1']
                            info_row['purple_ad_perk2'] = p['stats']['perk2']
                            info_row['purple_ad_perk3'] = p['stats']['perk3']
                            info_row['purple_ad_perk4'] = p['stats']['perk4']
                            info_row['purple_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['purple_sup_champion'] = p['championId']
                            info_row['purple_sup_item0'] = p['stats']['item0']
                            info_row['purple_sup_item1'] = p['stats']['item1']
                            info_row['purple_sup_item2'] = p['stats']['item2']
                            info_row['purple_sup_item3'] = p['stats']['item3']
                            info_row['purple_sup_item4'] = p['stats']['item4']
                            info_row['purple_sup_item5'] = p['stats']['item5']
                            info_row['purple_sup_item6'] = p['stats']['item6']
                            info_row['purple_sup_kills'] = p['stats']['kills']
                            info_row['purple_sup_deaths'] = p['stats']['deaths']
                            info_row['purple_sup_assists'] = p['stats']['assists']
        #                     info_row['purple_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_sup_perk0'] = p['stats']['perk0']
                            info_row['purple_sup_perk1'] = p['stats']['perk1']
                            info_row['purple_sup_perk2'] = p['stats']['perk2']
                            info_row['purple_sup_perk3'] = p['stats']['perk3']
                            info_row['purple_sup_perk4'] = p['stats']['perk4']
                            info_row['purple_sup_perk5'] = p['stats']['perk5']

            except:
                time.sleep(3)
                m = watcher.match.by_id(region, player_match['gameId'])
                info_row['Player_name'] = player_name
                info_row['gameId'] = m['gameId']
                info_row['platformId'] = m['platformId']
                info_row['gameDuration'] = m['gameDuration']
                info_row['gameVersion'] = m['gameVersion']

                blue_team = m['teams'][0]
                purple_team = m['teams'][1]
                info_row['blue_win'] = blue_team['win']
                info_row['blue_firstBlood'] = blue_team['firstBlood']
                info_row['blue_firstTower'] = blue_team['firstTower']
                info_row['blue_firstInhibitor'] = blue_team['firstInhibitor']
                info_row['blue_firstBaron'] = blue_team['firstBaron']
                info_row['blue_firstDragon'] = blue_team['firstDragon']
                info_row['blue_firstRiftHerald'] = blue_team['firstRiftHerald']
                info_row['blue_towerKills'] = blue_team['towerKills']
                info_row['blue_inhibitorKills'] = blue_team['inhibitorKills']
                info_row['blue_baronKills'] = blue_team['baronKills']
                info_row['blue_dragonKills'] = blue_team['dragonKills']
                info_row['blue_riftHeraldKills'] = blue_team['riftHeraldKills']

                info_row['purple_firstBlood'] = purple_team['firstBlood']
                info_row['purple_firstTower'] = purple_team['firstTower']
                info_row['purple_firstInhibitor'] = purple_team['firstInhibitor']
                info_row['purple_firstBaron'] = purple_team['firstBaron']
                info_row['purple_firstDragon'] = purple_team['firstDragon']
                info_row['purple_firstRiftHerald'] = purple_team['firstRiftHerald']
                info_row['purple_towerKills'] = purple_team['towerKills']
                info_row['purple_inhibitorKills'] = purple_team['inhibitorKills']
                info_row['purple_baronKills'] = purple_team['baronKills']
                info_row['purple_dragonKills'] = purple_team['dragonKills']
                info_row['purple_riftHeraldKills'] = purple_team['riftHeraldKills']

                ps = 0
                participants = m['participants']
                for p in participants:
                    print(ps)
                    ps = ps+1
                    if p['teamId'] == 100:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['blue_top_champion'] = p['championId']
                            info_row['blue_top_item0'] = p['stats']['item0']
                            info_row['blue_top_item1'] = p['stats']['item1']
                            info_row['blue_top_item2'] = p['stats']['item2']
                            info_row['blue_top_item3'] = p['stats']['item3']
                            info_row['blue_top_item4'] = p['stats']['item4']
                            info_row['blue_top_item5'] = p['stats']['item5']
                            info_row['blue_top_item6'] = p['stats']['item6']
                            info_row['blue_top_kills'] = p['stats']['kills']
                            info_row['blue_top_deaths'] = p['stats']['deaths']
                            info_row['blue_top_assists'] = p['stats']['assists']
        #                     info_row['blue_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_top_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_top_perk0'] = p['stats']['perk0']
                            info_row['blue_top_perk1'] = p['stats']['perk1']
                            info_row['blue_top_perk2'] = p['stats']['perk2']
                            info_row['blue_top_perk3'] = p['stats']['perk3']
                            info_row['blue_top_perk4'] = p['stats']['perk4']
                            info_row['blue_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['blue_jungle_champion'] = p['championId']
                            info_row['blue_jungle_item0'] = p['stats']['item0']
                            info_row['blue_jungle_item1'] = p['stats']['item1']
                            info_row['blue_jungle_item2'] = p['stats']['item2']
                            info_row['blue_jungle_item3'] = p['stats']['item3']
                            info_row['blue_jungle_item4'] = p['stats']['item4']
                            info_row['blue_jungle_item5'] = p['stats']['item5']
                            info_row['blue_jungle_item6'] = p['stats']['item6']
                            info_row['blue_jungle_kills'] = p['stats']['kills']
                            info_row['blue_jungle_deaths'] = p['stats']['deaths']
                            info_row['blue_jungle_assists'] = p['stats']['assists']
        #                     info_row['blue_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
    #                         info_row['blue_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
    #                         info_row['blue_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
    #                         info_row['blue_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_jungle_perk0'] = p['stats']['perk0']
                            info_row['blue_jungle_perk1'] = p['stats']['perk1']
                            info_row['blue_jungle_perk2'] = p['stats']['perk2']
                            info_row['blue_jungle_perk3'] = p['stats']['perk3']
                            info_row['blue_jungle_perk4'] = p['stats']['perk4']
                            info_row['blue_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['blue_mid_champion'] = p['championId']
                            info_row['blue_mid_item0'] = p['stats']['item0']
                            info_row['blue_mid_item1'] = p['stats']['item1']
                            info_row['blue_mid_item2'] = p['stats']['item2']
                            info_row['blue_mid_item3'] = p['stats']['item3']
                            info_row['blue_mid_item4'] = p['stats']['item4']
                            info_row['blue_mid_item5'] = p['stats']['item5']
                            info_row['blue_mid_item6'] = p['stats']['item6']
                            info_row['blue_mid_kills'] = p['stats']['kills']
                            info_row['blue_mid_deaths'] = p['stats']['deaths']
                            info_row['blue_mid_assists'] = p['stats']['assists']
        #                     info_row['blue_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_mid_perk0'] = p['stats']['perk0']
                            info_row['blue_mid_perk1'] = p['stats']['perk1']
                            info_row['blue_mid_perk2'] = p['stats']['perk2']
                            info_row['blue_mid_perk3'] = p['stats']['perk3']
                            info_row['blue_mid_perk4'] = p['stats']['perk4']
                            info_row['blue_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['blue_ad_champion'] = p['championId']
                            info_row['blue_ad_item0'] = p['stats']['item0']
                            info_row['blue_ad_item1'] = p['stats']['item1']
                            info_row['blue_ad_item2'] = p['stats']['item2']
                            info_row['blue_ad_item3'] = p['stats']['item3']
                            info_row['blue_ad_item4'] = p['stats']['item4']
                            info_row['blue_ad_item5'] = p['stats']['item5']
                            info_row['blue_ad_item6'] = p['stats']['item6']
                            info_row['blue_ad_kills'] = p['stats']['kills']
                            info_row['blue_ad_deaths'] = p['stats']['deaths']
                            info_row['blue_ad_assists'] = p['stats']['assists']
        #                     info_row['blue_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['blue_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['blue_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['blue_ad_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['blue_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_ad_perk0'] = p['stats']['perk0']
                            info_row['blue_ad_perk1'] = p['stats']['perk1']
                            info_row['blue_ad_perk2'] = p['stats']['perk2']
                            info_row['blue_ad_perk3'] = p['stats']['perk3']
                            info_row['blue_ad_perk4'] = p['stats']['perk4']
                            info_row['blue_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['blue_sup_champion'] = p['championId']
                            info_row['blue_sup_item0'] = p['stats']['item0']
                            info_row['blue_sup_item1'] = p['stats']['item1']
                            info_row['blue_sup_item2'] = p['stats']['item2']
                            info_row['blue_sup_item3'] = p['stats']['item3']
                            info_row['blue_sup_item4'] = p['stats']['item4']
                            info_row['blue_sup_item5'] = p['stats']['item5']
                            info_row['blue_sup_item6'] = p['stats']['item6']
                            info_row['blue_sup_kills'] = p['stats']['kills']
                            info_row['blue_sup_deaths'] = p['stats']['deaths']
                            info_row['blue_sup_assists'] = p['stats']['assists']
        #                     info_row['blue_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']                   
        #                     info_row['blue_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']                 
        #                     info_row['blue_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_sup_perk0'] = p['stats']['perk0']
                            info_row['blue_sup_perk1'] = p['stats']['perk1']
                            info_row['blue_sup_perk2'] = p['stats']['perk2']
                            info_row['blue_sup_perk3'] = p['stats']['perk3']
                            info_row['blue_sup_perk4'] = p['stats']['perk4']
                            info_row['blue_sup_perk5'] = p['stats']['perk5']

                    if p['teamId'] == 200:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['purple_top_champion'] = p['championId']
                            info_row['purple_top_item0'] = p['stats']['item0']
                            info_row['purple_top_item1'] = p['stats']['item1']
                            info_row['purple_top_item2'] = p['stats']['item2']
                            info_row['purple_top_item3'] = p['stats']['item3']
                            info_row['purple_top_item4'] = p['stats']['item4']
                            info_row['purple_top_item5'] = p['stats']['item5']
                            info_row['purple_top_item6'] = p['stats']['item6']
                            info_row['purple_top_kills'] = p['stats']['kills']
                            info_row['purple_top_deaths'] = p['stats']['deaths']
                            info_row['purple_top_assists'] = p['stats']['assists']
        #                     info_row['purple_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['purple_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['purple_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['purple_top_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['purple_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_top_perk0'] = p['stats']['perk0']
                            info_row['purple_top_perk1'] = p['stats']['perk1']
                            info_row['purple_top_perk2'] = p['stats']['perk2']
                            info_row['purple_top_perk3'] = p['stats']['perk3']
                            info_row['purple_top_perk4'] = p['stats']['perk4']
                            info_row['purple_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['purple_jungle_champion'] = p['championId']
                            info_row['purple_jungle_item0'] = p['stats']['item0']
                            info_row['purple_jungle_item1'] = p['stats']['item1']
                            info_row['purple_jungle_item2'] = p['stats']['item2']
                            info_row['purple_jungle_item3'] = p['stats']['item3']
                            info_row['purple_jungle_item4'] = p['stats']['item4']
                            info_row['purple_jungle_item5'] = p['stats']['item5']
                            info_row['purple_jungle_item6'] = p['stats']['item6']
                            info_row['purple_jungle_kills'] = p['stats']['kills']
                            info_row['purple_jungle_deaths'] = p['stats']['deaths']
                            info_row['purple_jungle_assists'] = p['stats']['assists']
        #                     info_row['purple_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_jungle_perk0'] = p['stats']['perk0']
                            info_row['purple_jungle_perk1'] = p['stats']['perk1']
                            info_row['purple_jungle_perk2'] = p['stats']['perk2']
                            info_row['purple_jungle_perk3'] = p['stats']['perk3']
                            info_row['purple_jungle_perk4'] = p['stats']['perk4']
                            info_row['purple_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['purple_mid_champion'] = p['championId']
                            info_row['purple_mid_item0'] = p['stats']['item0']
                            info_row['purple_mid_item1'] = p['stats']['item1']
                            info_row['purple_mid_item2'] = p['stats']['item2']
                            info_row['purple_mid_item3'] = p['stats']['item3']
                            info_row['purple_mid_item4'] = p['stats']['item4']
                            info_row['purple_mid_item5'] = p['stats']['item5']
                            info_row['purple_mid_item6'] = p['stats']['item6']
                            info_row['purple_mid_kills'] = p['stats']['kills']
                            info_row['purple_mid_deaths'] = p['stats']['deaths']
                            info_row['purple_mid_assists'] = p['stats']['assists']
        #                     info_row['purple_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_mid_perk0'] = p['stats']['perk0']
                            info_row['purple_mid_perk1'] = p['stats']['perk1']
                            info_row['purple_mid_perk2'] = p['stats']['perk2']
                            info_row['purple_mid_perk3'] = p['stats']['perk3']
                            info_row['purple_mid_perk4'] = p['stats']['perk4']
                            info_row['purple_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['purple_ad_champion'] = p['championId']
                            info_row['purple_ad_item0'] = p['stats']['item0']
                            info_row['purple_ad_item1'] = p['stats']['item1']
                            info_row['purple_ad_item2'] = p['stats']['item2']
                            info_row['purple_ad_item3'] = p['stats']['item3']
                            info_row['purple_ad_item4'] = p['stats']['item4']
                            info_row['purple_ad_item5'] = p['stats']['item5']
                            info_row['purple_ad_item6'] = p['stats']['item6']
                            info_row['purple_ad_kills'] = p['stats']['kills']
                            info_row['purple_ad_deaths'] = p['stats']['deaths']
                            info_row['purple_ad_assists'] = p['stats']['assists']
        #                     info_row['purple_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_ad_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_ad_perk0'] = p['stats']['perk0']
                            info_row['purple_ad_perk1'] = p['stats']['perk1']
                            info_row['purple_ad_perk2'] = p['stats']['perk2']
                            info_row['purple_ad_perk3'] = p['stats']['perk3']
                            info_row['purple_ad_perk4'] = p['stats']['perk4']
                            info_row['purple_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['purple_sup_champion'] = p['championId']
                            info_row['purple_sup_item0'] = p['stats']['item0']
                            info_row['purple_sup_item1'] = p['stats']['item1']
                            info_row['purple_sup_item2'] = p['stats']['item2']
                            info_row['purple_sup_item3'] = p['stats']['item3']
                            info_row['purple_sup_item4'] = p['stats']['item4']
                            info_row['purple_sup_item5'] = p['stats']['item5']
                            info_row['purple_sup_item6'] = p['stats']['item6']
                            info_row['purple_sup_kills'] = p['stats']['kills']
                            info_row['purple_sup_deaths'] = p['stats']['deaths']
                            info_row['purple_sup_assists'] = p['stats']['assists']
        #                     info_row['purple_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_sup_perk0'] = p['stats']['perk0']
                            info_row['purple_sup_perk1'] = p['stats']['perk1']
                            info_row['purple_sup_perk2'] = p['stats']['perk2']
                            info_row['purple_sup_perk3'] = p['stats']['perk3']
                            info_row['purple_sup_perk4'] = p['stats']['perk4']
                            info_row['purple_sup_perk5'] = p['stats']['perk5']

            matches_info.append(info_row)
    except:
        continue
    
    

matches_df = pd.DataFrame(matches_info)
matches_df.to_excel('matches2.xlsx')


In [96]:
matches = pd.read_excel('matches.xlsx')
matches

,Player_name,gameId,platformId,gameDuration,gameVersion,blue_win,blue_firstBlood,blue_firstTower,blue_firstInhibitor,blue_firstBaron,...,purple_sup_damageDealtToTurrets,purple_sup_visionScore,purple_sup_goldEarned,purple_sup_totalMinionsKilled,purple_sup_perk0,purple_sup_perk1,purple_sup_perk2,purple_sup_perk3,purple_sup_perk4,purple_sup_perk5
0,Alphari,3431750971,NA1,1901,10.10.320.3039,Fail,False,False,False,False,...,1260.0,27.0,8748.0,22.0,8229.0,8226.0,8210.0,8236.0,8138.0,8105.0
1,Alphari,3431727575,NA1,1059,10.10.320.3039,Fail,False,False,False,False,...,2451.0,12.0,6891.0,130.0,8112.0,8139.0,8138.0,8135.0,8345.0,8352.0
2,Alphari,3431684703,NA1,1219,10.10.320.3039,Win,True,True,True,False,...,294.0,9.0,5349.0,12.0,8214.0,8226.0,8210.0,8237.0,8009.0,8017.0
3,Alphari,3390881604,NA1,1980,10.8.317.8137,Win,False,False,True,False,...,0.0,72.0,11203.0,81.0,8112.0,8126.0,8120.0,8135.0,8236.0,8233.0
4,Alphari,3390781578,NA1,2365,10.8.317.8137,Win,False,False,True,False,...,2831.0,68.0,12805.0,88.0,8229.0,8226.0,8233.0,8237.0,8313.0,8345.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23194,Brolia,1031694247,TR1,1452,10.12.325.6222,Win,True,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23195,Brolia,1031683540,TR1,1818,10.12.325.6222,Win,True,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23196,Brolia,1031673556,TR1,1200,10.12.325.6222,Win,True,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23197,Brolia,1031682948,TR1,1381,10.12.325.6222,Win,True,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
nona_matches = matches.dropna()
nona_matches

,Player_name,gameId,platformId,gameDuration,gameVersion,blue_win,blue_firstBlood,blue_firstTower,blue_firstInhibitor,blue_firstBaron,...,purple_sup_damageDealtToTurrets,purple_sup_visionScore,purple_sup_goldEarned,purple_sup_totalMinionsKilled,purple_sup_perk0,purple_sup_perk1,purple_sup_perk2,purple_sup_perk3,purple_sup_perk4,purple_sup_perk5
0,Alphari,3431750971,NA1,1901,10.10.320.3039,Fail,False,False,False,False,...,1260.0,27.0,8748.0,22.0,8229.0,8226.0,8210.0,8236.0,8138.0,8105.0
2,Alphari,3431684703,NA1,1219,10.10.320.3039,Win,True,True,True,False,...,294.0,9.0,5349.0,12.0,8214.0,8226.0,8210.0,8237.0,8009.0,8017.0
3,Alphari,3390881604,NA1,1980,10.8.317.8137,Win,False,False,True,False,...,0.0,72.0,11203.0,81.0,8112.0,8126.0,8120.0,8135.0,8236.0,8233.0
4,Alphari,3390781578,NA1,2365,10.8.317.8137,Win,False,False,True,False,...,2831.0,68.0,12805.0,88.0,8229.0,8226.0,8233.0,8237.0,8313.0,8345.0
8,Alphari,3390572756,NA1,2725,10.8.317.8137,Fail,True,False,False,False,...,560.0,57.0,14070.0,78.0,8214.0,8226.0,8233.0,8237.0,8347.0,8313.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23141,Brolia,1035338633,TR1,2192,10.13.326.4870,Win,False,True,False,True,...,5198.0,34.0,13136.0,180.0,8008.0,9111.0,9103.0,8014.0,8275.0,8236.0
23147,Brolia,1034757655,TR1,1357,10.13.326.4870,Fail,False,False,False,False,...,0.0,2.0,5464.0,53.0,8010.0,9111.0,9103.0,8014.0,8345.0,8347.0
23148,Brolia,1034678660,TR1,1504,10.13.326.4870,Fail,False,False,False,False,...,3131.0,4.0,8784.0,123.0,8010.0,8009.0,9103.0,8017.0,8345.0,8347.0
23149,Brolia,1034656079,TR1,1712,10.13.326.4870,Fail,False,True,False,True,...,2474.0,15.0,8767.0,154.0,8008.0,9111.0,9103.0,8014.0,8275.0,8236.0


In [98]:
nona_matches.replace(False, 0, inplace = True)
nona_matches.replace(['Fail','Win'], [0,1], inplace = True)
nona_matches.reset_index(inplace = True)

C:\Users\Johnson\Anaconda3\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [99]:
for t in ['blue', 'purple']:
    for p in ['top', 'jungle', 'mid', 'ad', 'sup']:        
#         print(nona_matches['{}_{}_deaths'.format(t,p)])
        nona_matches['{}_{}_champname'.format(t,p)] = 0
        nona_matches['{}_{}_deaths'.format(t,p)][nona_matches['{}_{}_deaths'.format(t,p)]==0] = 1
#         print(nona_matches['{}_{}_deaths'.format(t,p)])

C:\Users\Johnson\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Johnson\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Johnson\Anaconda3\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [100]:
np.unique(nona_matches['platformId'])

array(['BR1', 'EUW1', 'JP1', 'KR', 'NA1', 'RU', 'TR1'], dtype=object)

In [101]:
champion_division_dict = {"BR1":"pt_BR", "EUW1":"en_GB", "JP1":"ja_JP", "KR":"ko_KR", "NA1":"en_US", "RU":"ru_RU", "TR1":"tr_TR"}

In [102]:
platform_ver = []
for i in range(len(nona_matches)):
    combo = []
    combo.append(nona_matches['platformId'][i])
    combo.append(str(nona_matches['gameVersion'][i]).split('.',2)[0:2])
    platform_ver.append(combo)
platform_ver

[['NA1', ['10', '10']],
 ['NA1', ['10', '10']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '4']],
 ['NA1', ['10', '4']],
 ['NA1', ['9', '24']],
 ['NA1', ['9', '22']],
 ['NA1', ['9', '22']],
 ['NA1', ['9', '21']],
 ['NA1', ['9', '20']],
 ['NA1', ['9', '20']],
 ['NA1', ['9', '20']],
 ['NA1', ['9', '20']],
 ['NA1', ['9', '20']],
 ['NA1', ['10', '16']],
 ['NA1', ['10', '10']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '8']],
 ['NA1'

In [103]:
for i in range(len(platform_ver)):
    for j in range(i):
        if platform_ver[i] == platform_ver[j]:
            platform_ver[i] = 0

for i in range(len(platform_ver)):
    try:
        platform_ver.remove(0)
    except:
        break
platform_ver

[['NA1', ['10', '10']],
 ['NA1', ['10', '8']],
 ['NA1', ['10', '4']],
 ['NA1', ['9', '24']],
 ['NA1', ['9', '22']],
 ['NA1', ['9', '21']],
 ['NA1', ['9', '20']],
 ['NA1', ['10', '16']],
 ['NA1', ['10', '9']],
 ['NA1', ['10', '7']],
 ['NA1', ['10', '25']],
 ['NA1', ['10', '24']],
 ['NA1', ['10', '23']],
 ['NA1', ['10', '22']],
 ['NA1', ['10', '21']],
 ['NA1', ['10', '20']],
 ['NA1', ['10', '19']],
 ['NA1', ['10', '18']],
 ['NA1', ['10', '15']],
 ['NA1', ['10', '14']],
 ['NA1', ['10', '11']],
 ['NA1', ['10', '6']],
 ['NA1', ['10', '3']],
 ['NA1', ['10', '2']],
 ['NA1', ['10', '1']],
 ['NA1', ['10', '5']],
 ['NA1', ['9', '16']],
 ['NA1', ['9', '15']],
 ['NA1', ['9', '14']],
 ['NA1', ['9', '13']],
 ['NA1', ['9', '12']],
 ['NA1', ['9', '11']],
 ['NA1', ['10', '13']],
 ['NA1', ['10', '12']],
 ['NA1', ['9', '23']],
 ['NA1', ['9', '19']],
 ['NA1', ['9', '18']],
 ['NA1', ['9', '17']],
 ['NA1', ['9', '10']],
 ['NA1', ['9', '5']],
 ['NA1', ['9', '2']],
 ['NA1', ['8', '24']],
 ['NA1', ['9', '9']],

In [104]:
api_key = 'RGAPI-3a2b6329-891f-473a-a9a1-de939b86bdc6'
watcher = LolWatcher(api_key)
ver_region_champ = []

for i in range(len(platform_ver)):
    combo = []
    ver = platform_ver[i][1]
    version = ver[0] + '.' + ver[1] + '.1'
    print(version)
    
    region = platform_ver[i][0]
    print(region)
    champ_list = watcher.data_dragon.champions(version, region)
#     print(champ_list)    
    champ_dict = {}
    for key in champ_list['data']:
        row = champ_list['data'][key]
        champ_dict[row['key']] = row['id']
    
    combo.append(version)
    combo.append(region)
    combo.append(champ_dict)
    ver_region_champ.append(combo)
# ver_region_champ  


10.10.1
NA1
10.8.1
NA1
10.4.1
NA1
9.24.1
NA1
9.22.1
NA1
9.21.1
NA1
9.20.1
NA1
10.16.1
NA1
10.9.1
NA1
10.7.1
NA1
10.25.1
NA1
10.24.1
NA1
10.23.1
NA1
10.22.1
NA1
10.21.1
NA1
10.20.1
NA1
10.19.1
NA1
10.18.1
NA1
10.15.1
NA1
10.14.1
NA1
10.11.1
NA1
10.6.1
NA1
10.3.1
NA1
10.2.1
NA1
10.1.1
NA1
10.5.1
NA1
9.16.1
NA1
9.15.1
NA1
9.14.1
NA1
9.13.1
NA1
9.12.1
NA1
9.11.1
NA1
10.13.1
NA1
10.12.1
NA1
9.23.1
NA1
9.19.1
NA1
9.18.1
NA1
9.17.1
NA1
9.10.1
NA1
9.5.1
NA1
9.2.1
NA1
8.24.1
NA1
9.9.1
NA1
9.8.1
NA1
9.6.1
NA1
10.25.1
EUW1
10.14.1
EUW1
10.13.1
EUW1
10.12.1
EUW1
10.23.1
EUW1
10.18.1
EUW1
10.7.1
EUW1
10.5.1
EUW1
10.4.1
EUW1
10.3.1
EUW1
10.2.1
EUW1
10.1.1
EUW1
10.22.1
EUW1
10.21.1
EUW1
10.11.1
EUW1
10.9.1
EUW1
10.8.1
EUW1
10.15.1
EUW1
9.22.1
EUW1
9.21.1
EUW1
9.9.1
EUW1
9.6.1
EUW1
9.5.1
EUW1
9.4.1
EUW1
9.7.1
EUW1
10.24.1
EUW1
10.20.1
EUW1
10.19.1
EUW1
9.3.1
EUW1
10.10.1
EUW1
10.16.1
EUW1
9.24.1
EUW1
9.18.1
EUW1
9.17.1
EUW1
9.15.1
EUW1
9.14.1
EUW1
9.13.1
EUW1
9.12.1
EUW1
9.1.1
EUW1
8.24.1
EUW1
9.10.1


In [105]:
for i in range(len(nona_matches)):
    for j in range(len(ver_region_champ)):
        if str(nona_matches['gameVersion'][i]).split('.')[0:2] == ver_region_champ[j][0].split('.')[0:2] and nona_matches['platformId'][i] == ver_region_champ[j][1]:
            print(nona_matches['gameVersion'][i])
            print(ver_region_champ[j][0])
            print(ver_region_champ[j][1])
            
            for t in ['blue', 'purple']:
                for p in ['top', 'jungle', 'mid', 'ad', 'sup']:
                    nona_matches['{}_{}_champname'.format(t,p)][i] = ver_region_champ[j][2][str(int(nona_matches['{}_{}_champion'.format(t,p)][i]))]

10.10.320.3039
10.10.1
NA1
10.10.320.3039
10.10.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.4.308.9400
10.4.1
NA1
10.4.308.9400
10.4.1
NA1
9.24.300.6382
9.24.1
NA1
9.22.296.5720
9.22.1
NA1
9.22.296.5720
9.22.1
NA1
9.21.294.813
9.21.1
NA1
9.20.292.2452
9.20.1
NA1
9.20.292.2452
9.20.1
NA1
9.20.292.2452
9.20.1
NA1
9.20.292.2452
9.20.1
NA1
9.20.292.2452
9.20.1
NA1
10.16.330.9186
10.16.1
NA1
10.10.320.3039
10.10.1
NA1
10.9.318.6057
10.9.1
NA1
10.9.318.6057
10.9.1
NA1
10.9.318.6057
10.9.1
NA1
10.9.318.6057
10.9.1
NA1
10.9.318.6057
10.9.1
NA1

C:\Users\Johnson\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Johnson\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Johnson\Anaconda3\lib\site-packages\pandas\core\series.py:1024: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers


10.9.318.6057
10.9.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.8137
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.317.3760
10.8.1
NA1
10.8.316.7615
10.8.1
NA1
10.8.316.7615
10.8.1
NA1
10.8.316.7615
10.8.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1

10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.21.339.2173
10.21.1
NA1
10.19.335.4706
10.19.1
NA1
10.18.333.8889
10.18.1
NA1
10.16.330.9186
10.16.1
NA1
10.16.330.9186
10.16.1
NA1
10.16.330.9186
10.16.1
NA1
10.7.314.9802
10.7.1
NA1
10.7.314.9802
10.7.1
NA1
10.6.314.4405
10.6.1
NA1
10.6.314.4405
10.6.1
NA1
10.6.314.4405
10.6.1
NA1
10.6.314.4405
10.6.1
NA1
10.6.313.8894
10.6.1
NA1
10.6.313.8894
10.6.1
NA1
10.6.313.8894
10.6.1
NA1
10.6.313.8894
10.6.1
NA1
10.5.311.166
10.5.1
NA1
10.5.311.166
10.5.1
NA1
10.5.311.166
10.5.1
NA1
10.5.311.166
10.5.1
NA1
10.5.311.166
10.5.1
NA1
10.5.311.166
10.5.1
NA1
9.16.284.8446
9.16.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.283.4011
9.15.1
NA1
9.15.

10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.350.1724
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.25.348.1797
10.25.1
NA1
10.24.345.8390
10.24.1
NA1
1

...

10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.24.345.8390
10.24.1
BR1
10.23.343.2581
10.23.1
BR1
10.23.343.2581
10.23.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.22.341.643
10.22.1
BR1
10.21.339.2173
10.21.1
BR1
10.20.338.336
10.20.1
BR1
10.20.337.6669
10.20.1
BR1
10.20.337.6669
10.20.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.19.336.4199
10.19.1
BR1
10.18.333.8889
10.18.1
BR1
10.18.333.8889
10.18.1
BR1
10.25.348

10.20.338.336
10.20.1
RU
10.16.330.9186
10.16.1
RU
10.16.330.9186
10.16.1
RU
10.16.330.9186
10.16.1
RU
10.16.330.9186
10.16.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.330.344
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.15.328.8759
10.15.1
RU
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1

10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.350.1724
10.25.1
JP1
10.25.348.1797
10.25.1
JP1
10.25.348.1797
10.25.1
JP1
10.24.345.8390
10.24.1
JP1
10.24.345.8390
10.24.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.23.343.2581
10.23.1
JP1
10.21.339.2173
10.21.1
JP1
10.21.339.2173
10.21.1
JP1
10.19.336.4199
10.19.1
JP1
10.11.322.2991
10.11.1
JP1
10.9.318.6057
10.9.1
JP1
10.9.318.6057
10.9.1
JP1
10.8.317.8137
10.8.1
JP1
10.8.317.8137
10.8.1
JP1
10.8.317.8137
10.8.1
JP1
10.8.317.81

10.18.333.8889
10.18.1
JP1
10.18.333.8889
10.18.1
JP1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.15.328.8759
10.15.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.14.327.8505
10.14.1
TR1
10.13.326.4870
10.13.1
TR1
10.13.326.4870
10.13.1
TR1
10.13.326.4870
10.13.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
1

10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.16.330.9186
10.16.1
TR1
10.8.317.8137
10.8.1
TR1
10.8.317.8137
10.8.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.25.350.1724
10.25.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.24.345.8390
10.24.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.23.343.2581
10.23.1
TR1
10.22.341.643
10.22.1
TR1
10.22.

In [106]:
nona_matches

,index,Player_name,gameId,platformId,gameDuration,gameVersion,blue_win,blue_firstBlood,blue_firstTower,blue_firstInhibitor,...,blue_top_champname,blue_jungle_champname,blue_mid_champname,blue_ad_champname,blue_sup_champname,purple_top_champname,purple_jungle_champname,purple_mid_champname,purple_ad_champname,purple_sup_champname
0,0,Alphari,3431750971,NA1,1901,10.10.320.3039,0,0.0,0.0,0.0,...,Udyr,Ekko,Zed,Jhin,Syndra,Riven,Kayn,Veigar,Jinx,Morgana
1,2,Alphari,3431684703,NA1,1219,10.10.320.3039,1,1.0,1.0,1.0,...,Gangplank,Udyr,Ekko,Cassiopeia,Thresh,Riven,Elise,Annie,Caitlyn,Yuumi
2,3,Alphari,3390881604,NA1,1980,10.8.317.8137,1,0.0,0.0,1.0,...,Riven,Udyr,Ivern,Varus,Leona,Vayne,JarvanIV,Swain,Xayah,Zed
3,4,Alphari,3390781578,NA1,2365,10.8.317.8137,1,0.0,0.0,1.0,...,Malphite,Udyr,Zoe,Yasuo,Camille,Talon,Graves,Katarina,Twitch,Brand
4,8,Alphari,3390572756,NA1,2725,10.8.317.8137,0,1.0,0.0,0.0,...,Gangplank,Udyr,Talon,Teemo,Brand,Malzahar,Rengar,Garen,Jhin,Lux
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7816,23141,Brolia,1035338633,TR1,2192,10.13.326.4870,1,0.0,1.0,0.0,...,Quinn,Alistar,Singed,Rakan,MonkeyKing,Sylas,Ahri,MissFortune,Olaf,Xayah
7817,23147,Brolia,1034757655,TR1,1357,10.13.326.4870,0,0.0,0.0,0.0,...,Riven,Rakan,Garen,Alistar,Graves,Lux,Velkoz,Xayah,LeeSin,Ezreal
7818,23148,Brolia,1034678660,TR1,1504,10.13.326.4870,0,0.0,0.0,0.0,...,Yasuo,Rakan,Sett,Thresh,JarvanIV,Lissandra,Zoe,Xayah,Kayn,Ezreal
7819,23149,Brolia,1034656079,TR1,1712,10.13.326.4870,0,0.0,1.0,0.0,...,Darius,Lulu,Fiora,Rakan,Shyvana,Diana,Syndra,Vayne,Trundle,Xayah


In [107]:
nona_matches.to_excel('matches_final.xlsx')

In [ ]:
# champions
champions_version = watcher.data_dragon.versions_for_region(region)['n']['champion']
# print(champions_version)
champ_list = watcher.data_dragon.champions(champions_version)

champ_dict = {}
for key in champ_list['data']:
    row = champ_list['data'][key]
    champ_dict[row['key']] = row['id']

champ_dict

In [ ]:
# items
items_version = watcher.data_dragon.versions_for_region(region)['n']['item']
items_list = watcher.data_dragon.items(items_version)

items_dict = {}
for key in items_list['data']:
    row = items_list['data'][key]
    items_dict[key] = row['name']

items_dict

In [ ]:
# runes
runes_version = watcher.data_dragon.versions_for_region(region)['n']['rune']
runes_list = watcher.data_dragon.runes_reforged(runes_version)
# print(runes_list[0]['slots'][0]['runes'][0]['id'])

runes_path_dict = {}
runes_dict = {}

for key in runes_list:
    runes_path_dict[key['id']] = key['name']

for key in runes_list:
    for slot in key['slots']:
        for runes in slot['runes']:
            runes_dict[runes['id']] = runes['name']

runes_dict
# items_dict

# For dear 采辰<3:

In [ ]:
api_key = 'RGAPI-112e5812-7a43-46a4-87d6-d1940a2b0862'
watcher = LolWatcher(api_key)

region = 'na1'
player_name = 'Alphari'
print(region)
print(player_name)

player = watcher.summoner.by_name(region, player_name)
print(player)
player_matches = watcher.match.matchlist_by_account(region, player['accountId'])['matches']
# player_matches[0]
m = watcher.match.by_id(region, player_matches[17]['gameId'])
m

In [ ]:
teams = []
teams_info = driver.find_elements(By.XPATH, "//div[@class='inline-content'][1]//table[@class='wikitable2 standings']//tr")
for t_f in teams_info[6:len(teams_info)]:
    team_info = t_f.text.replace('\u2060\u2060','').split(' ', 1)[1].split('/n')
    team_names.append(team_info[0])
    
    team_record = team_info[1].split(' ')
    teams_win.append(team_record[0])
    teams_lose.append(team_record[2])
    teams_winrate.append(team_record[3])
    teams_streak.append(team_record[4])